In [12]:
import pandas as pd
import numpy as np
import wfdb
import ast
import json
import os

In [13]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '../ptb-xl/'
sampling_rate=100

# Major Labels
- NORM
- STTC
- MI
- HYP
- CD

# Load up the default database

In [14]:
# load and convert annotation data
files = pd.read_csv(os.path.join(path,'ptbxl_database.csv'), index_col='ecg_id')
files.scp_codes = files.scp_codes.apply(lambda x: ast.literal_eval(x))
files[['age', 'sex', 'height', 'weight', 'scp_codes']].head(20)

age  sex  height  weight                                   scp_codes
ecg_id                                                                       
1       56.0    1     NaN    63.0    {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
2       19.0    0     NaN    70.0                {'NORM': 80.0, 'SBRAD': 0.0}
3       37.0    1     NaN    69.0                  {'NORM': 100.0, 'SR': 0.0}
4       24.0    0     NaN    82.0                  {'NORM': 100.0, 'SR': 0.0}
5       19.0    1     NaN    70.0                  {'NORM': 100.0, 'SR': 0.0}
6       18.0    1     NaN    58.0                  {'NORM': 100.0, 'SR': 0.0}
7       54.0    0     NaN    83.0                  {'NORM': 100.0, 'SR': 0.0}
8       48.0    0     NaN    95.0      {'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
9       55.0    0     NaN    70.0                  {'NORM': 100.0, 'SR': 0.0}
10      22.0    1     NaN    56.0                  {'NORM': 100.0, 'SR': 0.0}
11      20.0    1     NaN    57.0                {'NORM': 80.0, 'SARRH': 0.0}
12      43.0    1     NaN    44.0                {'NORM': 80.0, 'SBRAD': 0.0}
13      58.0    1     NaN    54.0                  {'NORM': 100.0, 'SR': 0.0}
14      19.0    1     NaN    58.0                  {'NORM': 100.0, 'SR': 0.0}
15      17.0    1     NaN    67.0               {'NORM': 100.0, 'SARRH': 0.0}
16      49.0    0     NaN    79.0                  {'NORM': 100.0, 'SR': 0.0}
17      56.0    0     NaN     NaN  {'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
18      56.0    0     NaN     NaN                             {'AFLT': 100.0}
19      20.0    0     NaN    85.0                  {'NORM': 100.0, 'SR': 0.0}
20      56.0    0     NaN     NaN               {'AFLT': 100.0, 'ABQRS': 0.0}

In [15]:
print(f'# of Height: {files.height.count()}')
print(f'# of Weight: {files.weight.count()}')
print(f'# of Age: {files.age.count()}')
print(f'# of Sex: {files.sex.count()}')


# of Height: 6974
# of Weight: 9421
# of Age: 21799
# of Sex: 21799


# Check which normal codes are less than 100 percent confidence, remove them

In [16]:
def less_than_100_check(row):
    codes = row['scp_codes']
    if 'NORM' in codes and codes['NORM'] < 100:
        return 1
    else:
        return 0
    
removal_mask = files.apply(less_than_100_check, axis=1).values
filtered_files = files[removal_mask == 0]
print(f"Number of normal codes with 100% confidence: {len(filtered_files)}")

Number of normal codes with 100% confidence: 19457


# Apply high level diagnostic codes for each of the classes

In [17]:
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
filtered_files['diagnostic_superclass'] = filtered_files.scp_codes.apply(aggregate_diagnostic)
filtered_files['diagnostic_superclass'].head(20)

/tmp/ipykernel_2550811/1254813600.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_files['diagnostic_superclass'] = filtered_files.scp_codes.apply(aggregate_diagnostic)


ecg_id
1     [NORM]
3     [NORM]
4     [NORM]
5     [NORM]
6     [NORM]
7     [NORM]
8       [MI]
9     [NORM]
10    [NORM]
13    [NORM]
14    [NORM]
15    [NORM]
16    [NORM]
17        []
18        []
19    [NORM]
20        []
21    [NORM]
22    [STTC]
23        []
Name: diagnostic_superclass, dtype: object

# Count how many of each class are present

In [18]:
diagnostic_superclass = filtered_files['diagnostic_superclass'].value_counts()
print(diagnostic_superclass)
classes = diagnostic_superclass.index
counts = diagnostic_superclass.values
frame = pd.DataFrame([co for co in zip(classes, counts)], columns=['class', 'counts'])

diagnostic_superclass
[NORM]                 7004
[MI]                   2532
[STTC]                 2400
[CD]                   1708
[CD, MI]               1297
[HYP, STTC]             781
[STTC, MI]              599
[HYP]                   535
[CD, STTC]              471
[]                      411
[HYP, STTC, MI]         361
[HYP, CD]               300
[CD, STTC, MI]          223
[HYP, MI]               183
[NORM, CD]              143
[HYP, CD, MI]           117
[HYP, STTC, CD]         109
[CD, HYP, STTC, MI]      99
[HYP, CD, STTC]          84
[HYP, CD, STTC, MI]      53
[NORM, STTC]             22
[CD, STTC, HYP]          18
[CD, STTC, HYP, MI]       4
[NORM, HYP]               2
[NORM, CD, STTC]          1
Name: count, dtype: int64


## Remove classes below the count of 20, as well as ones which contain NORM as well as other labels.

In [19]:
# Convert the index to a list and drop rows based on a condition
def convert_and_check(row):
    classes = row['class']
    count = row['counts']
    if 'NORM' in classes and len(classes) > 1:
        return 1    # Drop
    elif len(classes) == 0:
        return 1
    elif count < 20:
        return 1
    else:
        return 0

drop_indices = frame.apply(convert_and_check, axis=1).values
filtered_classes = frame[drop_indices == 0]
print(filtered_classes)

                  class  counts
0                [NORM]    7004
1                  [MI]    2532
2                [STTC]    2400
3                  [CD]    1708
4              [CD, MI]    1297
5           [HYP, STTC]     781
6            [STTC, MI]     599
7                 [HYP]     535
8            [CD, STTC]     471
10      [HYP, STTC, MI]     361
11            [HYP, CD]     300
12       [CD, STTC, MI]     223
13            [HYP, MI]     183
15        [HYP, CD, MI]     117
16      [HYP, STTC, CD]     109
17  [CD, HYP, STTC, MI]      99
18      [HYP, CD, STTC]      84
19  [HYP, CD, STTC, MI]      53


## Remove the undesired classes from the dataset

In [20]:
def all_values_match(x, y):
    '''Check if all values in x are in y'''
    if len(x) != len(y):
        return 0
    else:
        for value in x:
            if value not in y:
                return 0
    return 1

def in_class_list(row):
    '''Check if the diagnostic superclass is in the filtered classes'''
    try:
        for allowable_classes in filtered_classes['class'].values:
            if all_values_match(allowable_classes, row['diagnostic_superclass']):
                return 1
        return 0
    
    except:
        print(row['diagnostic_superclass'])
        print(filtered_classes['class'].values)

keep_indices = filtered_files.apply(in_class_list, axis=1)
filtered_files = filtered_files[keep_indices == 1]


# Label 0 for normal, 1 for abnormal

In [22]:
def is_normal(row):
    '''Check if the diagnostic superclass is normal'''
    if 'NORM' in row['diagnostic_superclass']:
        return 1
    else:
        return 0

In [23]:
norm_col = filtered_files.apply(is_normal, axis=1)

filtered_files['NORM'] = norm_col
filtered_files['ABNORM'] = (filtered_files['NORM'] == 0).astype(int)

In [24]:
filtered_files

patient_id    age  sex  height  weight  nurse  site      device  \
ecg_id                                                                    
1          15709.0   56.0    1     NaN    63.0    2.0   0.0   CS-12   E   
3          20372.0   37.0    1     NaN    69.0    2.0   0.0   CS-12   E   
4          17014.0   24.0    0     NaN    82.0    2.0   0.0   CS-12   E   
5          17448.0   19.0    1     NaN    70.0    2.0   0.0   CS-12   E   
6          19005.0   18.0    1     NaN    58.0    2.0   0.0   CS-12   E   
...            ...    ...  ...     ...     ...    ...   ...         ...   
21833      17180.0   67.0    1     NaN     NaN    1.0   2.0  AT-60    3   
21834      20703.0  300.0    0     NaN     NaN    1.0   2.0  AT-60    3   
21835      19311.0   59.0    1     NaN     NaN    1.0   2.0  AT-60    3   
21836       8873.0   64.0    1     NaN     NaN    1.0   2.0  AT-60    3   
21837      11744.0   68.0    0     NaN     NaN    1.0   2.0  AT-60    3   

             recording_date  \
ecg_id                        
1       1984-11-09 09:17:34   
3       1984-11-15 12:49:10   
4       1984-11-15 13:44:57   
5       1984-11-17 10:43:15   
6       1984-11-28 13:32:13   
...                     ...   
21833   2001-05-31 09:14:35   
21834   2001-06-05 11:33:39   
21835   2001-06-08 10:30:27   
21836   2001-06-09 18:21:49   
21837   2001-06-11 16:43:01   

                                                   report  ... burst_noise  \
ecg_id                                                     ...               
1                  sinusrhythmus periphere niederspannung  ...         NaN   
3                              sinusrhythmus normales ekg  ...         NaN   
4                              sinusrhythmus normales ekg  ...         NaN   
5                              sinusrhythmus normales ekg  ...         NaN   
6                              sinusrhythmus normales ekg  ...         NaN   
...                                                   ...  ...         ...   
21833   ventrikulÄre extrasystole(n) sinustachykardie ...  ...         NaN   
21834   sinusrhythmus lagetyp normal qrs(t) abnorm    ...  ...         NaN   
21835   sinusrhythmus lagetyp normal t abnorm in anter...  ...         NaN   
21836   supraventrikulÄre extrasystole(n) sinusrhythmu...  ...         NaN   
21837   sinusrhythmus p-sinistrocardiale lagetyp norma...  ...         NaN   

       electrodes_problems extra_beats pacemaker  strat_fold  \
ecg_id                                                         
1                      NaN         NaN       NaN           3   
3                      NaN         NaN       NaN           5   
4                      NaN         NaN       NaN           3   
5                      NaN         NaN       NaN           4   
6                      NaN         NaN       NaN           4   
...                    ...         ...       ...         ...   
21833                  NaN         1ES       NaN           7   
21834                  NaN         NaN       NaN           4   
21835                  NaN         NaN       NaN           2   
21836                  NaN        SVES       NaN           8   
21837                  NaN         NaN       NaN           9   

                      filename_lr                filename_hr  \
ecg_id                                                         
1       records100/00000/00001_lr  records500/00000/00001_hr   
3       records100/00000/00003_lr  records500/00000/00003_hr   
4       records100/00000/00004_lr  records500/00000/00004_hr   
5       records100/00000/00005_lr  records500/00000/00005_hr   
6       records100/00000/00006_lr  records500/00000/00006_hr   
...                           ...                        ...   
21833   records100/21000/21833_lr  records500/21000/21833_hr   
21834   records100/21000/21834_lr  records500/21000/21834_hr   
21835   records100/21000/21835_lr  records500/21000/21835_hr   
21836   records100/21000/21836_lr  records500/21000/21836_hr   
21837   reco

In [25]:
len(filtered_files[filtered_files['NORM'] == 0])/len(filtered_files)

0.6289861214111664

In [26]:
filtered_files.to_json(os.path.join(path, 'updated_ptbxl_database.json'))